In [1]:
import numpy as np
import matplotlib.pyplot as plt

from evaluation import eval_multi, MODE_ADVERSARY, MODE_RANDOM, MODE_ADV_TASK_DIVERSITY
%load_ext autoreload
%autoreload 2

In [2]:
d = 5
n_gen_context = 20 # Sample from the action set
T = 1000 # Timesteps
n_sim = 3 # Number of Simulations
m = 3
n_task = 5
assert m<=d, "m >= d"
assert T>=d**2, "T < d**2"
alpha_set = [1.0] # Bonus for UCB
v_set = [1.0] # Bonus for TS
alpha_PHE_set = [1.0] # Bonus for PHE
tau_1_PEGE_set = [T//2] # EXR length for PEGE
is_multi = False
adv_const = 1 # scaling factor for adversary task generation
PMA_const = 1 # scaling factor for exploration probability
PMA_lr_const = 1 # scaling factor for the learning rate
PMA_n_expert = 20 # Expert set size for PMA
input_dict = {
    "d":d,
    "n_gen_context":n_gen_context,
    "T":T,
    "n_sim":n_sim,
    "rho":0.5,
    "noise_std":1,
    "seed":None,
    "output":True,
    "params_set":alpha_set,
    "m":m,
    "n_task":n_task,
    "adv_const":adv_const,
    "PMA_const":PMA_const,
    "PMA_lr_const":PMA_lr_const,
    "mode":MODE_ADV_TASK_DIVERSITY,
    "PMA_n_expert":PMA_n_expert,
}

In [5]:
input_dict["name"] = "UCB"
input_dict["params_set"] = alpha_set
if is_multi:
    best_UCB = eval_multi(input_dict)
else:
    best_UCB = eval(input_dict)

UCB Simulation 1, N_gen_ctx=20, d=5, alpha=1.000


  0%|          | 0/5 [00:00<?, ?it/s]

> /home/thangduong/Serena/evaluation.py(184)gen_params()
    182             n_revealed = min(n_revealed+1,m)
    183         import pdb; pdb.set_trace()
--> 184         low_rank_B = np.copy(B)
    185         low_rank_B[:, n_revealed:] = 0
    186         theta = _gen_params_from_B(low_rank_B, m)

1
1
1
1
1
1
1
1
1
1
2024-05-24 23:53:33 Adversary reveal prob: q=1, n_revealed=1/3, theta=[-0.19534908  0.5592406   0.52847115 -0.08410052  0.20759694]


 20%|██        | 1/5 [00:56<03:45, 56.33s/it]

> /home/thangduong/Serena/evaluation.py(184)gen_params()
    182             n_revealed = min(n_revealed+1,m)
    183         import pdb; pdb.set_trace()
--> 184         low_rank_B = np.copy(B)
    185         low_rank_B[:, n_revealed:] = 0
    186         theta = _gen_params_from_B(low_rank_B, m)

1
*** NameError: name 'revealed_new' is not defined
*** NameError: name 'revealed_new' is not defined
1
1


 20%|██        | 1/5 [01:33<06:12, 93.16s/it]


In [ ]:
input_dict["name"] = "PEGE"
input_dict["params_set"] = tau_1_PEGE_set
if is_multi:
    best_UCB = eval_multi(input_dict)
else:
    best_UCB = eval(input_dict)

In [ ]:
input_dict["name"] = "TS"
input_dict["params_set"] = v_set
if is_multi:
    best_TS = eval_multi(input_dict)
else:
    best_TS = eval(input_dict)

In [ ]:
input_dict["name"] = "PHE"
input_dict["params_set"] = alpha_PHE_set
if is_multi:
    best_PHE = eval_multi(input_dict)
else:
    best_PHE = eval(input_dict)

In [ ]:
if is_multi:
    mean_UCB = best_UCB['regrets']
    std_UCB = np.std(mean_UCB, axis=0)
    mean_UCB = np.mean(mean_UCB, axis=0)
    mean_TS = best_TS['regrets']
    std_TS = np.std(mean_TS, axis=0)
    mean_TS = np.mean(mean_TS, axis=0)
    mean_PHE = best_PHE['regrets']
    std_PHE = np.std(mean_PHE, axis=0)
    mean_PHE = np.mean(mean_PHE, axis=0)

    plt.plot(np.arange(1,n_task+1), mean_UCB, '-.', label='UCB')
    plt.fill_between(np.arange(1,n_task+1), mean_UCB-std_UCB, mean_UCB+std_UCB, alpha=0.5)
    plt.plot(np.arange(1,n_task+1), mean_TS, '-.', label='TS')
    plt.fill_between(np.arange(1,n_task+1), mean_TS-std_TS, mean_TS+std_TS, alpha=0.5)
    plt.plot(np.arange(1,n_task+1), mean_PHE, '-.', label='PHE')
    plt.fill_between(np.arange(1,n_task+1), mean_PHE-std_PHE, mean_PHE+std_PHE, alpha=0.5)
else:
    mean_UCB = np.mean(best_UCB['regrets'], axis=0)
    std_UCB = np.std(best_UCB['regrets'], axis=0)
    mean_TS = np.mean(best_TS['regrets'], axis=0)
    std_TS = np.std(best_TS['regrets'], axis=0)
    mean_PHE = np.mean(best_PHE['regrets'], axis=0)
    std_PHE = np.std(best_PHE['regrets'], axis=0)

    plt.plot(np.arange(1,T+1), mean_UCB, '-.', label='UCB')
    plt.fill_between(np.arange(1,T+1), mean_UCB-std_UCB, mean_UCB+std_UCB, alpha=0.5)
    plt.plot(np.arange(1,T+1), mean_TS, '-.', label='TS')
    plt.fill_between(np.arange(1,T+1), mean_TS-std_TS, mean_TS+std_TS, alpha=0.5)
    plt.plot(np.arange(1,T+1), mean_PHE, '-.', label='PHE')
    plt.fill_between(np.arange(1,T+1), mean_PHE-std_PHE, mean_PHE+std_PHE, alpha=0.5)

plt.legend()
plt.show()
plt.close()